# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [138]:
# import libraries
import pandas as pd
import numpy as np
import nltk
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
import pickle


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/labuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/labuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/labuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [139]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('disaster_pipeline', engine)
display(df.head())
X = df['message']
print(df.iloc[1][2])
Y = df.iloc[:,4:]
df.shape

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Cyclone nan fini osinon li pa fini


(26177, 40)

### 2. Write a tokenization function to process your text data

In [140]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_regex, text)
    for url in urls:
        text = text.replace(url, "urlplaceholder")
    tokens = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()
        
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

tokenize('the words are bright')

['the', 'word', 'are', 'bright']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [141]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3

/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

In [143]:
msg = ['Fire in my heart, peace in my soul, water in my eyes']
test_output = pipeline.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related']


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [144]:
y_pred = pipeline.predict(X_test)
display(y_pred)
display(y_test)
print(y_pred.shape)
print(y_test.shape)

array([[1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
11503,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12169,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19469,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
5427,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5178,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8953,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
6205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4257,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
14561,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
4087,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


(6545, 36)
(6545, 36)


In [145]:
for pred, orig, column in zip(y_pred.transpose(), y_test.values.transpose(), y_test.columns):
    print(column)
    print(classification_report(orig, pred))
    
overall_accuracy = (y_pred == y_test).mean().mean()
print("Overall accuracy is {}".format(overall_accuracy))

related
              precision    recall  f1-score   support

           0       0.61      0.35      0.44      1520
           1       0.82      0.93      0.87      4981
           2       0.75      0.14      0.23        44

   micro avg       0.79      0.79      0.79      6545
   macro avg       0.72      0.47      0.52      6545
weighted avg       0.77      0.79      0.77      6545

request
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      5440
           1       0.85      0.34      0.48      1105

   micro avg       0.88      0.88      0.88      6545
   macro avg       0.86      0.66      0.71      6545
weighted avg       0.87      0.88      0.86      6545

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6521
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      6545
   macro avg       0.50      0.50      0.50      654

/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


As we can see from above output scores  of columns like shops, hospitals, tools, offers, aid_centres has very poor metrics( 0 for many) to identify positive cases. This model needs lot of improvement.

### 6. Improve your model
Use grid search to find better parameters. 

In [147]:
params = pipeline.get_params()
scorer = make_scorer(f1_score)

print(params)


parameters = {
    'vect__ngram_range': ((1, 1), (1, 2), (2,2)),
    'vect__max_df': (0.5, 1.0),
    'clf__estimator__max_depth': (25,  100, None),
    'clf__estimator__min_samples_split': (6, 10, 25, 50), 
    'clf__estimator__n_estimators': [10, 50],
}

cv = GridSearchCV (pipeline, param_grid= parameters, verbose = 2, n_jobs = 3)

{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x1ab6f11ae8>, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=Fals

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [148]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  6.1min
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 25.0min
[Parallel(n_jobs=3)]: Done 359 tasks      | elapsed: 121.7min
[Parallel(n_jobs=3)]: Done 432 out of 432 | elapsed: 167.1min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=3,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2), (2, 2)), 'vect__max_df': (0.5, 1.0), 'clf__estimator__max_depth': (25, 100, None), 'clf__estimator__min_samples_split': (6, 10, 25, 50), 'clf__estimator__n_estimators': [10, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [149]:
print(cv.best_score_)
cv.best_params_

0.23604319478402608


{'clf__estimator__max_depth': None,
 'clf__estimator__min_samples_split': 25,
 'clf__estimator__n_estimators': 50,
 'vect__max_df': 0.5,
 'vect__ngram_range': (1, 2)}

In [150]:
y_pred = cv.predict(X_test)

In [151]:
for pred, orig, column in zip(y_pred.transpose(), y_test.values.transpose(), y_test.columns):
    print(column)
    print(classification_report(orig, pred))
    
overall_accuracy = (y_pred == y_test).mean().mean()
print("Overall accuracy is {}".format(overall_accuracy))

related
              precision    recall  f1-score   support

           0       0.69      0.25      0.36      1520
           1       0.80      0.97      0.88      4981
           2       0.50      0.05      0.08        44

   micro avg       0.79      0.79      0.79      6545
   macro avg       0.66      0.42      0.44      6545
weighted avg       0.77      0.79      0.75      6545

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5440
           1       0.86      0.46      0.60      1105

   micro avg       0.90      0.90      0.90      6545
   macro avg       0.88      0.72      0.77      6545
weighted avg       0.89      0.90      0.88      6545

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6521
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      6545
   macro avg       0.50      0.50      0.50      654

/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [128]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_regex, text)
    for url in urls:
        text = text.replace(url, "urlplaceholder")
    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
        
    tokens = [lemmatizer.lemmatize(word).lower().strip() for word in tokens if word not in stop_words]

    return tokens

tokenize('the words are bright')


pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

for pred, orig, column in zip(y_pred.transpose(), y_test.values.transpose(), y_test.columns):
    print(column)
    print(classification_report(orig, pred))
    
overall_accuracy = (y_pred == y_test).mean().mean()
print("Overall accuracy is {}".format(overall_accuracy))



/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3

/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


related
              precision    recall  f1-score   support

           0       0.64      0.37      0.47      1583
           1       0.82      0.93      0.87      4916
           2       0.00      0.00      0.00        46

   micro avg       0.79      0.79      0.79      6545
   macro avg       0.49      0.43      0.45      6545
weighted avg       0.77      0.79      0.77      6545

request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5424
           1       0.81      0.41      0.55      1121

   micro avg       0.88      0.88      0.88      6545
   macro avg       0.85      0.70      0.74      6545
weighted avg       0.88      0.88      0.87      6545

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        23

   micro avg       1.00      1.00      1.00      6545
   macro avg       0.50      0.50      0.50      654

/Users/labuser/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Slight improvement in model performance.

### 9. Export your model as a pickle file

In [153]:
with open('data/model.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [154]:
!ls -lrt

total 27232
-rw-r--r--@ 1 labuser  staff       613 Sep 10  2018 README.md
drwxr-xr-x@ 3 labuser  staff        96 Sep 11  2018 models
drwxr-xr-x@ 5 labuser  staff       160 Jul 25 11:35 app
-rw-r--r--  1 labuser  staff  12775424 Jul 27 15:34 InsertDatabaseName.db
-rw-r--r--@ 1 labuser  staff     44522 Jul 27 15:36 ETL Pipeline Preparation.ipynb
drwxr-xr-x@ 6 labuser  staff       192 Aug  3 12:55 data
-rw-r--r--@ 1 labuser  staff    161817 Aug  3 17:47 ML Pipeline Preparation.ipynb
